In [14]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torchvision.transforms as transforms
import torchvision.models as models
import optuna

In [15]:
# Set random seeds for reproducibility
torch.manual_seed(42)

In [16]:
# DEVICE
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [17]:
data_path = "D:\SH\dataset"
print(os.listdir(data_path))

['Ayrshire', 'Brown_Swiss', 'Deoni', 'Gir', 'Hallikar', 'Holstein_Friesian', 'Jersey', 'Kangayam', 'Kankrej', 'Nagpuri', 'Pulikulam', 'Rathi', 'Red_Dane', 'Sahiwal', 'Tharparkar']


In [18]:
images = []
labels = []

for breed_folder in os.listdir(data_path):
    breed_path = os.path.join(data_path, breed_folder)
    if os.path.isdir(breed_path):
        for image_name in os.listdir(breed_path):
            img_path = os.path.join(breed_path, image_name)
            img = cv2.imread(img_path, cv2.IMREAD_COLOR) 
            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, (224, 224))  # Bigger size for ResNet
                images.append(img)
                labels.append(breed_folder)

X = np.array(images, dtype="uint8")  # keep raw images
y = np.array(labels)

In [19]:
# Encode labels
le = LabelEncoder()
y = le.fit_transform(y)
num_classes = len(np.unique(y))

In [20]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [21]:
# CUSTOM DATASET
class CustomDataset(Dataset):
    def __init__(self, features, labels, transform=None):
        self.features = features
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        img = self.features[idx].astype(np.uint8)
        if self.transform:
            img = self.transform(img)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return img, label

In [22]:
# Evaluation
def evaluate(model, loader, device):
    model.eval()
    top1_correct, top3_correct, total = 0, 0, 0
    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, pred_top1 = outputs.topk(1, dim=1)
            _, pred_top3 = outputs.topk(3, dim=1)
            total += labels.size(0)
            top1_correct += (pred_top1.squeeze() == labels).sum().item()
            top3_correct += sum([labels[i] in pred_top3[i] for i in range(labels.size(0))])
    return 100 * top1_correct / total, 100 * top3_correct / total

In [23]:
# Optuna Objective Function
# -----------------------------
def objective(trial):
    # Hyperparameters to tune
    lr = trial.suggest_float("lr", 1e-5, 1e-3, log=True)
    optimizer_name = trial.suggest_categorical("optimizer", ["AdamW", "Adam", "SGD"])
    weight_decay = trial.suggest_float("weight_decay", 1e-4, 1e-1, log=True)
    freeze_layers = trial.suggest_categorical("freeze_layers", ["layer2+", "layer3+", "layer4+"])

    # Augmentation hyperparams
    random_flip = trial.suggest_float("random_flip", 0.0, 0.7)
    rotation = trial.suggest_int("rotation", 0, 25)
    color_jitter = trial.suggest_float("color_jitter", 0.0, 0.2)
    random_erasing_prob = trial.suggest_float("random_erasing_prob", 0.0, 0.3)

    # Transforms
    train_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomResizedCrop(224, scale=(0.85, 1.0)),
        transforms.RandomHorizontalFlip(p=random_flip),
        transforms.RandomRotation(rotation),
        transforms.ColorJitter(
            brightness=color_jitter,
            contrast=color_jitter,
            saturation=color_jitter,
            hue=color_jitter/4
        ),
        transforms.RandomAffine(degrees=0, translate=(0.05, 0.05), shear=5),
        transforms.RandomPerspective(distortion_scale=0.1, p=0.3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
        transforms.RandomErasing(p=random_erasing_prob, scale=(0.02, 0.15), ratio=(0.3, 3.3))
    ])

    test_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
    ])

    train_dataset = CustomDataset(X_train, y_train, transform=train_transform)
    test_dataset = CustomDataset(X_test, y_test, transform=test_transform)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)

    # Model
    model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
    model.fc = nn.Linear(model.fc.in_features, num_classes)

    # Freeze layers based on trial
    for name, param in model.named_parameters():
        param.requires_grad = True
        if freeze_layers == "layer2+" and "layer1" in name:
            param.requires_grad = False
        elif freeze_layers == "layer3+" and ("layer1" in name or "layer2" in name):
            param.requires_grad = False
        elif freeze_layers == "layer4+" and ("layer1" in name or "layer2" in name or "layer3" in name):
            param.requires_grad = False

    model = model.to(device)

    # Loss & Optimizer
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    if optimizer_name == "AdamW":
        optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, weight_decay=weight_decay)
    elif optimizer_name == "Adam":
        optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, weight_decay=weight_decay)
    else:
        optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, momentum=0.9, weight_decay=weight_decay)

    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

    # Training loop (small for Optuna tuning)
    epochs = 10
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for imgs, labels_batch in train_loader:
            imgs, labels_batch = imgs.to(device), labels_batch.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels_batch)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        avg_loss = running_loss / len(train_loader)
        scheduler.step(avg_loss)

    # Evaluate
    top1, _ = evaluate(model, test_loader, device)
    return top1  # Optuna maximizes top-1 accuracy


In [24]:
# Run Optuna Study
# -----------------------------
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

# Save best hyperparameters
best_params = study.best_trial.params
print("Best Hyperparameters:")
for k, v in best_params.items():
    print(f"{k}: {v}")

[I 2025-09-22 13:48:14,714] A new study created in memory with name: no-name-c493a660-3bec-4076-862d-8df6257cc6b0
[I 2025-09-22 13:58:49,775] Trial 0 finished with value: 77.46666666666667 and parameters: {'lr': 2.401085770442434e-05, 'optimizer': 'AdamW', 'weight_decay': 0.00032450386837293976, 'freeze_layers': 'layer2+', 'random_flip': 0.3192947469307138, 'rotation': 24, 'color_jitter': 0.19005424975035304, 'random_erasing_prob': 0.1625762613076371}. Best is trial 0 with value: 77.46666666666667.
[I 2025-09-22 14:06:37,839] Trial 1 finished with value: 76.13333333333334 and parameters: {'lr': 0.0006694904605317828, 'optimizer': 'SGD', 'weight_decay': 0.0013137322255709748, 'freeze_layers': 'layer2+', 'random_flip': 0.6913223474921092, 'rotation': 2, 'color_jitter': 0.01974671521967706, 'random_erasing_prob': 0.2187558507643673}. Best is trial 0 with value: 77.46666666666667.
[I 2025-09-22 14:11:05,421] Trial 2 finished with value: 81.2 and parameters: {'lr': 8.516437928175004e-05, 'o

Best Hyperparameters:
lr: 8.516437928175004e-05
optimizer: AdamW
weight_decay: 0.0022419282598442726
freeze_layers: layer3+
random_flip: 0.31746700173524217
rotation: 3
color_jitter: 0.03414274380074951
random_erasing_prob: 0.1991334544419178


In [25]:
# Full Training with Best Hyperparameters
# -----------------------------
# Train transforms using best hyperparams
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomResizedCrop(224, scale=(0.85, 1.0)),
    transforms.RandomHorizontalFlip(p=best_params["random_flip"]),
    transforms.RandomRotation(best_params["rotation"]),
    transforms.ColorJitter(
        brightness=best_params["color_jitter"],
        contrast=best_params["color_jitter"],
        saturation=best_params["color_jitter"],
        hue=best_params["color_jitter"]/4
    ),
    transforms.RandomAffine(degrees=0, translate=(0.05,0.05), shear=5),
    transforms.RandomPerspective(distortion_scale=0.1, p=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
    transforms.RandomErasing(p=best_params["random_erasing_prob"], scale=(0.02, 0.15), ratio=(0.3, 3.3))
])

test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

train_dataset = CustomDataset(X_train, y_train, transform=train_transform)
test_dataset = CustomDataset(X_test, y_test, transform=test_transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)

# Build model with best freeze layers
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, num_classes)
freeze_layers = best_params["freeze_layers"]
for name, param in model.named_parameters():
    param.requires_grad = True
    if freeze_layers == "layer2+" and "layer1" in name:
        param.requires_grad = False
    elif freeze_layers == "layer3+" and ("layer1" in name or "layer2" in name):
        param.requires_grad = False
    elif freeze_layers == "layer4+" and ("layer1" in name or "layer2" in name or "layer3" in name):
        param.requires_grad = False
model = model.to(device)

# Optimizer & Scheduler
optimizer_name = best_params["optimizer"]
lr = best_params["lr"]
weight_decay = best_params["weight_decay"]
if optimizer_name == "AdamW":
    optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, weight_decay=weight_decay)
elif optimizer_name == "Adam":
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, weight_decay=weight_decay)
else:
    optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, momentum=0.9, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)


In [26]:
# Full Retraining Loop
full_epochs = 20
for epoch in range(full_epochs):
    model.train()
    running_loss = 0.0
    for imgs, labels_batch in train_loader:
        imgs, labels_batch = imgs.to(device), labels_batch.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{full_epochs}], Loss: {avg_loss:.4f}")
    scheduler.step(avg_loss)

Epoch [1/20], Loss: 1.8958
Epoch [2/20], Loss: 1.2621
Epoch [3/20], Loss: 1.0782
Epoch [4/20], Loss: 0.9468
Epoch [5/20], Loss: 0.8663
Epoch [6/20], Loss: 0.7998
Epoch [7/20], Loss: 0.7731
Epoch [8/20], Loss: 0.7444
Epoch [9/20], Loss: 0.7302
Epoch [10/20], Loss: 0.7009
Epoch [11/20], Loss: 0.6959
Epoch [12/20], Loss: 0.6990
Epoch [13/20], Loss: 0.6942
Epoch [14/20], Loss: 0.6927
Epoch [15/20], Loss: 0.6772
Epoch [16/20], Loss: 0.6734
Epoch [17/20], Loss: 0.6616
Epoch [18/20], Loss: 0.6621
Epoch [19/20], Loss: 0.6584
Epoch [20/20], Loss: 0.6546


In [27]:
# Evaluate Train & Test Accuracy
train_top1, train_top3 = evaluate(model, train_loader, device)
test_top1, test_top3 = evaluate(model, test_loader, device)
print(f"Train Set - Top-1 Accuracy: {train_top1:.2f}%, Top-3 Accuracy: {train_top3:.2f}%")
print(f"Test Set - Top-1 Accuracy: {test_top1:.2f}%, Top-3 Accuracy: {test_top3:.2f}%")

Train Set - Top-1 Accuracy: 98.73%, Top-3 Accuracy: 99.97%
Test Set - Top-1 Accuracy: 82.67%, Top-3 Accuracy: 93.07%


In [28]:
import pickle

In [29]:
# Save trained model
torch.save(model.state_dict(), "best_breed_model.pth")
print("Model saved as 'best_breed_model.pth'")


Model saved as 'best_breed_model.pth'


In [30]:
# Load model + prediction function
def load_model(model_path, num_classes, device):
    model = models.resnet18(weights=None)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    return model

def predict_breed(image_path, model, le, device):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Image not found: {image_path}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224,224))
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
    img_tensor = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(img_tensor)
        probs = torch.softmax(outputs, dim=1)
        top_prob, top_idx = torch.max(probs, dim=1)
    breed = le.inverse_transform(top_idx.cpu().numpy())[0]
    return breed, top_prob.item()


In [33]:
# Example Prediction
loaded_model = load_model("best_breed_model.pth", num_classes, device)
test_image_path = "D:\download.jpeg"
breed, confidence = predict_breed(test_image_path, loaded_model, le, device)
print(f"Predicted Breed: {breed} ")
#(Confidence: {confidence:.2f})

C:\Users\suraj\AppData\Local\Temp\ipykernel_36292\863495743.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=dev

Predicted Breed: Jersey 
